In [1]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import matplotlib.pyplot as plt 
from torchvision import transforms, datasets
import os 
from PIL import Image 

In [2]:
path = "PetImages"

### Preprocessing and splitting into training and testing datasets.

In [3]:
for pet_folder in os.listdir(path):
    for file in os.listdir(os.path.join(path, pet_folder)):
        image_file = os.path.join(path, pet_folder, file)
        if os.path.splitext(image_file)[1] == ".jpg":
            pass 
        else:
            os.remove(image_file)

In [4]:
transformations = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(40),
    transforms.Grayscale(1),
    transforms.ToTensor(),
    transforms.Normalize([0.5], 0.5)
])

dataset = datasets.ImageFolder(path, transform=transformations)

In [5]:
dataset_len = len(dataset)
dataset_len

24998

In [6]:
train_len, test_len = dataset_len-6000, 6000 
train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len])
batch_size = 50

train_set = torch.utils.data.DataLoader(dataset=train_set, shuffle=True, batch_size=batch_size)
test_set = torch.utils.data.DataLoader(dataset=test_set, shuffle=True, batch_size=batch_size)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device, ', device)

Using device,  cuda


### Building the Model.

In [7]:
class Model(nn.Module):
    def __init__(self) -> None:
        super(self, Model).__init__()

        #Convolutional layers.
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size=(5, 5), stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size=(5, 5), stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=(3, 3), padding=1)

        #Linear layer/sequential layer.
        self.fc1 = nn.Linear(in_features= 64 * 6 * 6, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=50)
        self.fc3 = nn.Linear(in_features=50, out_features=2)